# Verbatim mode

In [4]:
from braket.circuits import Circuit, Observable, Gate, Noise
from braket.aws import AwsDevice
import numpy as np
import datetime

In [2]:
now = datetime.datetime.now().strftime('%y%m%d%H%M%S')
my_bucket = "amazon-braket-hardware-testing" # the name of the bucket
my_prefix = "oqc-verbatim-" + now # the name of the folder in the bucket
s3_folder = (my_bucket, my_prefix)

# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-1")
# device = AwsDevice("arn:aws:braket:::device/qpu/rigetti/Aspen-11")
device = AwsDevice("arn:aws:braket:eu-west-2::device/qpu/oqc/Lucy")

In [3]:
# Available gates
for gate in device.properties.paradigm.nativeGateSet:
    print(gate)

rx
rz
cz
cphaseshift
xy


In [6]:
# verbatim circuit 
identity = Circuit().rz(0,np.pi).rz(0,np.pi)
circ1 = Circuit().add_verbatim_box(identity)
print(circ1)

# verbatim circuit with observables
identity = Circuit().rz(0,np.pi).rz(0,np.pi)
circ2 = Circuit().add_verbatim_box(identity)
circ2.expectation(Observable.X(), target=[0])
circ2.expectation(Observable.X(), target=[1])
print(circ2)

T  : |      0      |   1    |   2    |     3     |
                                                  
q0 : -StartVerbatim-Rz(3.14)-Rz(3.14)-EndVerbatim-

T  : |      0      |   1    |   2    |     3     |
T  : |      0      |   1    |   2    |     3     | Result Types |
                                                                 
q0 : -StartVerbatim-Rz(3.14)-Rz(3.14)-EndVerbatim-Expectation(X)-
      |                               |                          
q1 : -*************-------------------***********-Expectation(X)-

T  : |      0      |   1    |   2    |     3     | Result Types |


### Execute circuits

In [ ]:
# For OQC's device
# reference: https://github.com/aws/amazon-braket-schemas-python-staging/blob/main/src/braket/task_result/oqc_metadata_v1.py
result1 = device.run(circ1, s3_folder, shots=100, disable_qubit_rewiring=True).result()
compiled_program = result1.additional_metadata.OqcMetadata.executedProgram
print(compiled_program)
print(result.measurement_counts)

result2 = device.run(circ2, s3_folder, shots=100, disable_qubit_rewiring=True).result()
compiled_program = result2.additional_metadata.OqcMetadata.executedProgram
print(compiled_program)
print(result.measurement_counts)
print(result.values)

In [ ]:
# For Rigetti's device
# result = device.run(circ1, s3_folder, shots=100, disable_qubit_rewiring=True).result()
# compiled_program = result.additional_metadata.rigettiMetadata.compiledProgram
# print(compiled_program)
# print(result.measurement_counts)